# Run this cell first

In [ ]:
# this code enables the automated feedback. If you remove this, you won't get any feedback
# so don't delete this cell!
try:
  import AutoFeedback
except (ModuleNotFoundError, ImportError):
  !pip install git+https://github.com/abrown41/AutoFeedback@notebook
  import AutoFeedback

try:
  from testsrc import test_main
except (ModuleNotFoundError, ImportError):
  !pip install "git+https://github.com/autofeedback-exercises/exercises.git@testpip#subdirectory=New-MTH4332/LennardJonesI"
  from testsrc import test_main

def runtest(tlist):
  import unittest
  from contextlib import redirect_stderr
  from os import devnull
  with redirect_stderr(open(devnull, 'w')):
    suite = unittest.TestSuite()
    for tname in tlist:
      suite.addTest(eval(f"test_main.UnitTests.{tname}"))
    runner = unittest.TextTestRunner()
    try:
      runner.run(suite)
    except AssertionError:
      pass


This next cell loads some code that allows us to use ase to run simulations of systems with a pair potential of your own design. In this notebook, that pair potential is the Lennard Jones potential. If you want to run simulations like those that we have performed here but using some other potential it may be useful for you to copy and reuse the code in this cell.

In [ ]:
import ase
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from ase.build import bulk,make_supercell
from ase.visualize import view
from ase.io import write
from ase.neighborlist import NeighborList
from ase.calculators.calculator import Calculator, all_changes
from ase.stress import full_3x3_to_voigt_6_stress
from ase.lattice.cubic import FaceCenteredCubic
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.langevin import Langevin
from ase.io.trajectory import Trajectory 

class pairwise_calculator(Calculator) :
  """Implementation of a very basic Lennard Jones Calculator"""

  implemented_properties = ['energy', 'energies', 'forces', 'free_energy']
  implemented_properties += ['stress', 'stresses']  # bulk properties
  default_parameters = {'rc': None,}
  nolabel = True

  def __init__(self, **kwargs ) :
    Calculator.__init__( self, **kwargs )
    if self.parameters.pairwise_e is None :
      ValueError("function for evaluating pairwise energies has not been set")
    # Setup a cutoff value so we can use a neighbour list
    if self.parameters.rc is None : 
      ValueError("cutoff for pairwise interactions should be set")
    # Setup stuff for neighbour list
    self.nl = None
    self.pairwise_e = self.parameters.pairwise_e
    self.parameters.pairwise_e = None

  def calculate( self, atoms=None, properties=None, system_changes=all_changes, ) :
    if properties is None : properties = self.implemented_properties

    Calculator.calculate(self, atoms, properties, system_changes)

    natoms = len(self.atoms)

    rc = self.parameters.rc
    # potential value at rc
    e0, f0 = self.pairwise_e( rc )     

    if self.nl is None or 'numbers' in system_changes:
        self.nl = NeighborList([rc / 2] * natoms, self_interaction=False, bothways=True )
    
    self.nl.update(self.atoms)

    positions = self.atoms.positions
    cell = self.atoms.cell

    energies = np.zeros(natoms)
    forces = np.zeros((natoms, 3))
    stresses = np.zeros((natoms, 3, 3))

    for ii in range(natoms):
        neighbors, offsets = self.nl.get_neighbors(ii)
        cells = np.dot(offsets, cell)

        # pointing *towards* neighbours
        distance_vectors = positions[neighbors] + cells - positions[ii]

        r = np.sqrt( (distance_vectors ** 2).sum(1) )
        pairwise_energies, pairwise_forces = self.pairwise_e( r )
        pairwise_energies[r > rc] = 0.0
        pairwise_forces[r > rc] = 0.0

        pairwise_forces = pairwise_forces[:, np.newaxis] * distance_vectors
        energies[ii] += 0.5 * pairwise_energies.sum()  # atomic energies
        forces[ii] += pairwise_forces.sum(axis=0)
        stresses[ii] += 0.5 * np.dot(pairwise_forces.T, distance_vectors)

    # no lattice, no stress
    if self.atoms.cell.rank == 3:
        stresses = full_3x3_to_voigt_6_stress(stresses)
        self.results['stress'] = stresses.sum(axis=0) / self.atoms.get_volume()
        self.results['stresses'] = stresses / self.atoms.get_volume()

    energy = energies.sum()
    self.results['energy'] = energy
    self.results['energies'] = energies
    self.results['free_energy'] = energy
    self.results['forces'] = forces

# Introduction

This notebook explains how you can run molecular dynamics simulations of a set of interacting atoms using a piece of software called ASE (the atomistic simulation environment).  You can find documentation for running molecular dynamics using ASE [here](https://ase-lib.org/ase/md.html).  Alternatively, the following video gives you a brief introduction to these ideas.

In [1]:
%%HTML 
<iframe width="560" height="315" src="https://www.youtube.com/embed/IMEJv4n7s-c?si=OhqxkEnJs7zx1y6k" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

As always you will need to run the following cell in order to load the libraries that we need for the exercises before you start work on the exercises.  Notice, furthermore, that there is a second cell in the hidden section at the top of this notebook that loads many features that we need in order to get ASE to run properly.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats

# Setting the initial configuration using Atomistic Simulation Environment

In the assignments you have completed thus far you have written all the python code for doing your simulations from scratch.
Writing code in this way is not particularly common when you do graduate level research.  The simulations that are typically done
as part of a PhD project are far more complicated than the ones you have performed in this module.  We thus need to reuse code that
has been written by other people as there simply is not time to write all the simulation code from scratch.  You are going to learn to
use a package for doing molecular dynamics simulations called [atomistic simulation environment (ase)](https://wiki.fysik.dtu.dk/ase/)
in these exercises.  You are then going to use this package to complete the next assignment.

Your task in this exercise involves using the `FaceCenteredCubic` method from ase to create an object called `atoms`.  This object will
contain the initial positions of the atoms.  The atomic positions should be sat on an face centered cubic lattice that has the $[1,0,0]$
direction of the crystal aligned with the x-axis, the $[0,1,0]$ direction of the crystal aligned with the y-axis and the $[0,0,1]$ direction of the crystal
aligned with the z-axis.  Each of the atoms in the lattice should have the symbol 'Ar' and the minimal unit cell should be repeated
$3 \timees 3 \times 3$ times.  The structure should be set up so that there are pbc in all three directions.  The lattice constant should be set equal to $2^(2/3)$.

You then need to use the [set_masses method](https://wiki.fysik.dtu.dk/ase/ase/atoms.html) from `atoms` to set all the masses of the atoms equal to one.

Lastly, because we are planning to do molecular dynamics, you will need to give each atom an initial velocity.  You can do this using the
[MaxwellBoltzmannDistribution](https://wiki.fysik.dtu.dk/ase/ase/md.html) method from ASE.  You should use this method to set your atoms to have velocities
that are consistent with the ones they would be expected to have at a temperature at which $(k_B T) = 2.0$ (to set the temperature you need to use the depreciated temp argument to this function).

In completing this task you will likely find [this page](https://wiki.fysik.dtu.dk/ase/ase/lattice.html) of the ASE manual useful as well as the links above.


In [ ]:
runtest(['test_ase'])

# Setting up the calculator

In completing the previous exercise you learned how to setup the initial structure.  You are thus half way towards
running an MD simulation.  The other half that you need to learn is how to create code to calculate the energy of the system
and the forces on the individual atoms.  Once you have code to calculate the forces ASE can use the Verlet algorithm that you
learned for second of these assignments to propegate the initial positions and velocities with time.

In ASE the forces are calculated by a calculator method that is attached to the atoms object that you learned to create in the previous
exericse.  Code to attach this method to the atoms object and then run NVT molecular dynamics would look something like this:

```python
# Insert code from last exercise to create an atoms object and set masses and velocities here.

# Attach the method that should be used to calculate energies and forces to the atoms object
atoms.calc = pairwise_calculator( rc=4, pairwise_e=fff )

# And run 1000 steps of Langevin dynamics on the particles. We are running Langevin dynamics
# with k_B T = 2, a timestep of 0.005 and a thermostat friction of 1.0 here.
dyn = Langevin( atoms, 0.005, 2.0, 1.0 )
dyn.run(1000)
```

The calculator method that we are using in this input is one that I have written for you (there are many others that
form part of of the ASE package).  The code for this calculator is what is in the second hidden cell at the top of this notebook.   When you come to do the assignments you will also need to copy the contents of this cell to your python notebooks in order to use the functionality within it.

The `pairwise_calc` calculator that I have written for you allows you to write a function to calulate the pair potential that acts between the atoms.  This function is then passed to ASE and used in the `pairwise_calculator` object.  In extending the simulations that have been done here you can use any function you so choose for this role.  In other words, the energy for a pair of atoms separated by a distance $r$ can be $f(r)$ where $f$ is some function of your own choosing.

__To complete this repl exercise you need to complete the code in `main.py` so that it can be used to run molecular dynamics for a system of Lennard Jones particles.__  The energy of a pair of atoms that are separated by a distance $r_{ij}$ and that interact through the Lennard Jones potential is given by:

$$
V(r_{ij}) = 4\left[ \left(\frac{1}{r_{ij}}\right)^{12} - \left(\frac{1}{r_{ij}}\right)^6 \right]
$$

This quantity should be calculated and returned by the function called `fff` that you must complete.  This function should also return a second quantity, which is the value of $f_{ij}$ such that:

$$
F_{x_i} = -\frac{\partial V(r_{ij})}{\partial x_i} = f_{ij} x_{ij}
$$

In this expression the derivative of $V$ the left hand side is the derivative of the pair potential with respect the $x$ position of atom $i$ and $x_{ij}$ is the $x$-component of the vector that connects atom $i$ to atom $j$.  Notice that the $r$ that enters the expression for the Lennard Jones pontential is given by:

$$
r_{ij} = \sqrt{ (x_i - x_j)^2 + (y_i - y_j)^2 + (z_i - z_j)^2 }
$$

and that when you calculate the forces on the atoms you have to differentiate with respect to the atomic positions and __not__ the distance between the atoms.  Doing this differentiation involves using the chain rule.

The fact that the function `fff` is to be used to calculate the pairwise energies is passed to the `pairwise_calculator` method by setting the argument `pairwise_e` equal to `fff`. The other argument to the function `rc` is a cutoff that tells ASE that the potential should be assumed to be zero when atoms are separated by a distance that is greater than `rc`.  To pass this exercise `rc` must be set to 4.  However, you may find this
cutoff needs to be set to a larger (or smaller) value when you study your potential

__Notice that you will also need to copy the code for setting the masses, initial positions and initial velocities that you wrote in the last exercise to `main.py` to complete the exercise.__  The initial positions, number of atoms and temperature should be set as they were in that previous exercise.


In [ ]:
def fff(r):
   # Insert your code to calculate the Lennard Jones energy and forces here

   return e, f  # First argument should be energy and second should be force

# Insert code from last exercise to create an atoms object and set masses and velocities here.


# Attach the method that should be used to calculate energies and forces to the atoms object
atoms.calc = pairwise_calculator( rc=4, pairwise_e=fff )

# And run 1000 steps of Langevin dynamics on the particles. We are running Langevin dynamics
# with k_B T = 2, a timestep of 0.005 and a thermostat friction of 1.0 here.
dyn = Langevin( atoms, 0.005, 2.0, 1.0 )
dyn.run(1000)

In [ ]:
runtest(['test_forces', 'test_calculator'])

# Analysing the MD trajectories

The code in the cell for this exercise is very similar to the one you had in the previous exericse.
To complete this exericse you are thus going to have to consolidate what you learned in the last exercise
and copy the codes that you have just written for initialising the positions and velocities and running Langevin dynamics.  You can reuse the `fff` function that you have wrote in the previous exercise and use it to evaluate the Lennard Jones energies and forces.

As you have seen in previous exercises, just running molecular dynamics caclulations is pointless.  You also need to do some analysis of the trajectories that are generated and compute some averages.  As discussed in the video, when we use a code like ASE there are two ways we can do this analysis:

1. We can modify what is done in the main MD loop for ASE and calculate the quantities that interest us 'on the fly'.  When you wrote your MD and MC codes for the previous assignments this is what you did.

2. We can save the trajectory that ASE generates to a file.  We can then read this trajectory after the MD simulation is completed and analyse it.

Both of these methods involve interacting with the ASE [atoms object](https://ase-lib.org/ase/atoms.html).

In [2]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/ksDN7x7fC90?si=Ahaf1RcdAnw_4TYL" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

The other exercises in this notebook will teach you how to analyse the trajectory files that are output when you save this trajectory (this way makes it easier for me to autograde your work).  However, when you come to doing the assignments you may prefer to do analysis on the fly so I will show you how to modify ASE's MD loop here as well.

You can ensure that a function is called after every 10 steps of MD by using code like the one below:

```python
# Code to initalise the structure and velocites in atoms needs to go here.

# Code to setup the routines for calculating the energies and forces goes here.

# Now that we have setup our initial positions and our routines for calculating energies and forces
# we can setup an object that does will allow us to do as much Langevin dynamics as we want.
# When we do this Langevin dynamics we will be running simulations with k_B T = 2, a timestep of 0.005 and a thermostat friction of 1.0 here.
dyn = Langevin( atoms, 0.005, 2.0, 1.0 )

# We now create an empty list that is going to hold snapshots of data we are collecting from the trajectory
stats = []

# When this function is called the current value of the potential energy is appended to the array stats
def printenergy(stats = stats, a=atoms ):
    stats.append( a.get_potential_energy() )

# This command ensures that the function printenergy above is called every 10 seconds of MD
dyn.attach( printenergy, interval=10 )
# And run 1000 steps of Langevin dynamics
dyn.run(1000)
```

This code calculates the energy potential energy on every 10th MD step and appends these values to the list called stats.  To do this we use the `dyn.attach` method from ASE to tell it that it should call the function `printenergy` on every 10th MD step.  You can see that this function just appends the instantaneous atom to the list called stats.

To output the trajectory we do something similar.  The only difference is that we use a function that is already part of ASE and that is one of the methods in the Trajectory object.  The code to output the trajectory to a file called moldyn3.traj is as follows:

```python
# Code to initalise the structure and velocites in atoms needs to go here.

# Code to setup the routines for calculating the energies and forces goes here.

# Now that we have setup our initial positions and our routines for calculating energies and forces
# we can setup an object that does will allow us to do as much Langevin dynamics as we want.
# When we do this Langevin dynamics we will be running simulations with k_B T = 2, a timestep of 0.005 and a thermostat friction of 1.0 here.
dyn = Langevin( atoms, 0.005, 2.0, 1.0 )

# Create a trajectory object.  This opens the file on which we will output configurations
traj = Trajectory('moldyn3.traj', 'w', atoms)

# Attach the write method for the Trajectory object to the MD loop
dyn.attach( traj.write, interval=10 )

# And run 1000 steps of Langevin dynamics
dyn.run(1000)

# And close the trajectory file once we are done writing to it
traj.close()
```

Notice how this code uses the `dyn.attach` method to attach the `traj.write` method to the MD loop.  It is this that ensures that the trajectory is output to the file every 10 MD steps.

__Your task in this exercise is to use ASE to write an MD code.  Your MD code should save the values the potential energy took on every 20th step to a NumPy array called `energies`.  Your code should also output the positions of all the atoms on every 20th step to a file called `mytraj.traj`.__ The energies and forces should be calculated using the Lennard Jones potential with cutoff of 4 that you have encountered in previous exercises.  I will test your code by ensuring that the energies in the NumPy array `energies` are consistent with energies of the configurations that are output in `mytraj.traj`.


In [ ]:
# Insert code from last exercise to create an atoms object and set masses and velocities here.



# Attach the method that should be used to calculate energies and forces to the atoms object
atoms.calc = pairwise_calculator( rc=4, pairwise_e=fff )

# And run 1000 steps of Langevin dynamics on the particles.  Use the information in the instructions
# to attach a funtion to the dynamics that keeps track of the values the potential energy took
# during the simulation.  The values the energy took should be stored in a list called energies.
# You should also output the trajectory to a file called mytraj.traj.  The frequency with which
# you output the trajectory frames should be the same as the one with which you output energies.



In [ ]:
runtest(['test_energies'])

# Calculating the radial distribution function

In the previous exercise you learned how you can use `dyn.attach` to access the positions of the atoms.  In the remainder of this notebook you are going to learn two methods for analysing the trajectories that ASE generates.  You can perform these analysis methods by attaching a function to do the ananlysis on the fly using `dyn.attach`.  Alternatively, you can use the approach that I am using here, where you save the trajectory and then analyse it in postprocessing.  I am doing postprocessing here because it is easier for me to test your code as we will all be using the trajectory that you can load by running the commands in the following cell.

In [ ]:
from ase.io.trajectory import Trajectory
mytraj = Trajectory('https://raw.githubusercontent.com/autofeedback-exercises/exercises/main/New-MTH4332/LennardJonesI/trajectory.traj')

In this exercise you are going to calculate the radial distribution.  As discussed in the video below, this function describes how the density varies as a function of the distance from a refernece particle.  It is a measure of the probability of finding a particle at a distnace of $r$ away from a given reference reference particles, relative to that probability in the ideal
gas.

In [3]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/iEsD5Jufw7o?si=KD232r-Tw3bi-g24" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

To calculate the radial distirbution function you need to adapt the code that you have previously learned to write to estimate the histograms.  The first step in calculating this function is to divide the interval between 0 and some maximum distance `maxd` into `nbins` discrete bins.  Each of the distances in each of the frames of the trajectory are then assigned to the appropriate bin just as you have learned to ascribe other observables to bins in a histogram.

There is a crucial difference between the process of calculating a histogram that you have gone through in previous exercises and the process of calculating a radial distribution function (rdf).  When you have calculated histograms in previous exercises there has been exactly one sample to add to the histogram for each frame of your trajectory.  __When you calculate a radial distribution function from a trajectory of N atoms each frame gives you N(N-1) samples of the random variable__ as there are $N(N-1)$ distances between your $N$ particles.  The radial distribution function should, therefore, converge relatively quickly as you can quickly generate a lot of data to estimate it from.

Having said all that, you will hopefully find the process of calculating the (unormalised) rdf rather easy as the code that I am asking you to write here is very similar to the code for writing a histogram that you have written many times before.  To help you get started I have written the following piece of code below:

```python
# Do a loop over all the trajectory frames
for atoms in mytraj :
    # Calculate the distances between all pairs of atoms
    distances = atoms.get_all_distances( mic=True )
    # Do a double loop over all the distances
    for i in range(1,distances.shape[0]) :
        for j in range(0,i) :
            if  distances[i,j]>maxd : continue
            # You need to add code to accumulate the histogram here
```

The first loop in this code iterates over all the trajectory frames.  I then use the `atoms.get_all_distances` from ASE to calculate the distances between all the particles in the system.  This method returns
an $N \times N$ NumPy array in which element $i,j$ gives the distance between atom $i$ and atom $j$.  The two inner loops in the code above iterate over the lower triangle of this (symmetric) matrix.  You need to add code
to accumulate the histogram from the various `distance[i,j]` values that are iterated over.  Think carefully about the fact that the loop will iterate over only $N(N-1)/2$ of the distance elements of the matrix
`distances`.

Once you have accumulated your histogram you need to normalise it.  When normalising the rdf you need to divide by:

1. The number of frames in the trajectory.
2. The number of atoms in the system.
3. The number of atoms that you would expect to find in an ideal gas that has the same volume and density as the bin.

The third of these quantities is given by the following expression:

$$
N_b  = \frac{N}{V} \frac{4}{3} \pi \left[ (l+d)^3 - l^3 \right] 
$$

In this expression $l$ indicates the value at which the bin starts, $d$ is the width of the bins $N$ is the total number of atoms and $V$ is the volume of the box.  This expression is just the difference
in volume between a sphere with radius $l+d$ and a sphere with radius $l$ multiplied by the density of the medium.  You can get the volume of the box from the ASE atoms object by using the command `V=atoms.get_volume()`

To pass the assignment you need to generate a plot of the radial distribution function that you have calculated from the trajectory.  Your radial distribution function should be calculated up to a maximum distance of `maxd` and
should have `nbins` bins.  The value of the radial distribution in the ith bin should be plotted at the midpoint of the bin.  The x-axis label for your plot should be 'r / sigma' and the y-axis label should be 'g(r)'.

In [ ]:
# Set the maximum distances that you are going to plot out to
# and the number of bins to use when calculating the radial distribution function
maxd, nbins = 3, 150

# Do a loop over all the trajectory frames
for atoms in Trajectory('trajectory.traj') :
    # Calculate the distances between all pairs of atoms
    distances = atoms.get_all_distances( mic=True )
    # Do a double loop over all the distances
    for i in range(1,distances.shape[0]) :
        for j in range(0,i) :
            if  distances[i,j]>maxd : continue
            # You need to add code to accumulate the histogram here


# Now you need to add code to normalise and plot the radial distribution function


# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_rdf3'])

# Errors on radial distribution functions

When you calculate a radial distribution function from an MD simulation the result you get is an estimate.  Consequently, as with everything else you have learned to calculate from MD simulations,
you need to quote error bars on your estimate.  You can get these errors using the block averaging technique that we have seen elsewhere in this course.  Furthermore, when you work out the lengths
of the blocks that are needed you use the same techniques as we have used in previous exercises.

__In this exercise you are going to generate a estimate of the radial distribution function along with the erorrs on this estimate of the distribution.__  I would like you to spit the data in the trajectory
that I have provided into __five__ blocks.  You can calculate 5 separate estimates of the radial distribution function from these five blocks of data.  Each of these estimates of the radial distribution function
should be calculated to a maximum distance of `maxd` and should use `nbins` bins.

Once you have calculated and normalised the five rdfs using what you learned in the previous exercise you should set the variable called `average` equal to the average of these five estimates of the rdf.  The variable
called error should be set equal to the error for a 90% confidence limit on your estimate.  To set this array you will need to calculate the variance for each of the bins from your five estimates of the rdf.

The variabels `average` and `error` that you will set (and that I test) should be NumPy arrays.  I have included code at the end of the following cell that will plot the radial distribution function and the estimate of the errors for you.  Notice that I use `plt.fill_between` as the rdf is a continuous function.  We thus do what we did in previous exericses and display the part of the xy-plane that this function passes through as a shaded area.


In [ ]:
maxd, nbins = 3, 150
# This is the width of each of the bins in your histogram
delx = maxd / nbins
# Your code to calculate the five estimates of the radial distribution from the blocks goes here


# I have calculated the x-coordinates at which to plot the histogram here
xbins = np.zeros(nbins)
for i in range(nbins) : xbins[i] = (0.5+i)*delx

# The first of these variables (average) should be the average rdf from the 5 blocks
# The secould (error) should contain the errors on the estimates of the density in each bin.
# Your error should be indicative of a 95 % confidence limit
average =
error =

# This will draw a graph showing your radial distribution function.  Notice that the radial distribution
# function we are plotting here is not well converged.  None of the radial distribution functions in your final
# project should look like this.  The big spikes suggest there is a problem.
plt.fill_between( xbins, average - error, average + error )
plt.xlabel('r / sigma')
plt.ylabel('g(r)')
plt.plot()

# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_rdf1'])

# Calculating the FCC cubic parameter

The radial distribution provides indirect information on whether the average structure of the material resembles an ordered solid or a more disordered liquid.  As discussed in the following video, another way to make this distinction is to use an order parameter or symmetry function.  If you are modelling a system of $N$ atoms you can caluclate $N$ instances of the order parameter as you calculate one order parameter for each atom in the system.  The order parameter is computed from the vectors that connect the central atom to each of the atoms in its first coordination sphere.  Often times order parameters are designed to be one when the environment about the atom resembles the environment in an ideal, crystalline solid and zero if it does not.

You can read about the particular order parameter we are implementing here by reading the documentation [here](https://www.plumed.org/doc-master/user-doc/html/FCCUBIC/).

In [4]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/QHfmiSvSB4M?si=m0b0I45jZ4vXrPCo" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In this exercise you are going to learn to calculate an order parameter that can be used to detect whether the environment around an atom resmebles the environment in an ideal fcc crystal.  The functional form of this order parameter is as follows:

$$
s_i = \frac{1}{\sum_{j\ne i} \sigma(r_{ij})} \sum_{j\ne i} \sigma(r_{ij}) \left[ \frac{(x_{ij}y_{ij})^4 + (x_{ij}z_{ij})^4 + (y_{ij}z_{ij})^4}{r_{ij}^8} - \frac{27(x_{ij}y_{ij}z_{ij})^4}{r_{ij}^{12}} \right]
$$

The sums in this expression run over all the atoms in the syste.  $r_{ij}$ is the distance between atom $i$ and atom $j$ and $x_{ij}$, $y_{ij}$ and $z_{ij}$ are the $x$, $y$ and $z$ components of the vector that connects atom $i$ to atom $j$.  $\sigma(r_{ij})$ is a switching function that is 1 if $r_{ij}<1.5$ and 0 otherwise.  When this function is used to identify regions where the structure resembles the fcc crystal it is useally tranformed using the following expression:

$$
\kappa_i = \frac{80080s_i}{2717 + 16\times 27} + \frac{16(27-143)}{2717 + 16\times 27} 
$$

__Your task in this exercise is to write a function called `fcc_cubic` that calculates this order parameter.__ Your function should take an Atoms object called atoms from ASE as input.  It should calculate the function above for __all__ $N$ atoms in this input object and store this information in a NumPy array with $N$ elements.  Your function should then return this NumPy array.

In the following cell I have included some code to get you started.  The code that I have written used ASE to generate a 2D NumPy array called `distances` that contains the pairwise distances between all pairs of particles in the system.  I have also used ASE to generate a 3D NumPy array called `vecs` that contains the $x$, $y$ and $z$ components of all the vectors connecting distinct pairs of atoms.  To get the $x$, $y$ and $z$ components of the vector connecting atom i to atom j from this array you would use code like this:

```python
x, y, z = vecs[i,j,0], vecs[i,j,1], vecs[i,j,2]
```

I have also setup NumPy array called `order_p` that you can use to hold the values of all the order parameters that you will calculate.

In [ ]:
def fcc_cubic( atoms ) :
    N = len(atoms)
    order_p = np.zeros(N)
    distances = atoms.get_all_distances( mic=True )
    vecs = atoms.get_all_distances( mic=True, vector=True )
    # Your code goes here


    return order_p


In [ ]:
runtest(['test_fcc'])

# Calculating the free energy as a function of an order parameter

This exercise should revise ideas that you covered last week when you learned to calculate the free energy as a function of the total magnetisation for the 2D Ising model.  What I would like you to calculate is the free energy as a function of the order parameter.  Furthermore, I would like you to calculate an error on your estimate of this free energy using block averaging.

We covered how to estimate a free energy surface when we looked at the 2D Ising model.  The only difference from what you did last week and what you will do here is that last week you only had one magnetisation value from each trajectory frame.  This week you have N=the number of atoms values for the order parameter for each trajectory frame.  You can use all of these N values when you accumulate your histograms.

You should devide up the trajectory into five blocks as you did when you caluclated the errors on the radial distribution function.  One histogram should be calculated from each block of trajectory data.  These histograms should show the distribution of CV values between `minx` and `mixx` and this interval should be split into `nbins` histogram bins.

Once you have calculated your 5 estimates for the histogram you can calculate the overal average and the error on the estimate of the distribution. The error on the estimate of the distribution should be calculated for a 90% confidence limit.

The free energy can be calculated from your estimate of the distribution in the usual way.  The variable `fes` should be set equal to the final estimate of the free energy.  You should use error propegation (as you did in the previous exercise) to calculate the error on the free energy.  The variabel `error` should be set equal to this value of the error.  The tests check that these two variables have been set to the correct values.  At the end of the exercise I have included code that will generate a figure that shows the final estimate for the free energy as a function of the order parameter.  The width of the line in this figure is related to the error on your estimate of the this free energy.

In [ ]:
minx, maxx, nbins = -.5, 1.1, 70
# Your code to calculate the estimates of the histogram goes here

# You then need to calculate the average histogram and the errors on the histogram here

# And lastly you need to convert the histogram to a free energy surface here


# This variable should be set equal to your final estimate of the free energy as a function of the order parameter
fes =
# This variable should be set equla to the error on your estimate of the free energy as a function of the order parameter
error =


# This sets the coordinates at which the x values of the points in your free energy surface should be plotted
xbins = np.zeros( nbins )
for i in range(len(gat_average)) :
    xbins[i] = (i+1)*delx


# This will draw a graph showing your free energy surface. Notice that the free energy surface
# we are plotting here is not well converged.
plt.fill_between( xbins, fes - error , fes + error )
plt.xlabel('order parameter')
plt.ylabel('free energy')
plt.plot()

# This code is required for the Automated feedback, don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_rdf2'])